In [151]:
%load_ext autoreload
%autoreload 2

import sys
import os
import dotenv
from pathlib import Path

env_file = "../.env"

if os.path.exists(env_file):
    dotenv.load_dotenv(env_file, verbose=True)
    print("Loaded environment variables from .env file.")

cwd = os.getcwd()
# for some reason appending to PATH you need it to be string
sys.path.append(str(Path(cwd).parent / "src"))
hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

# from research_tools.gpu import get_gpus_available

# os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in get_gpus_available()])
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
# get rid of linker warnings that show up for some reason
os.environ["OTHER_LDFLAGS"] = "-w"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Loaded environment variables from .env file.


In [152]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert device == torch.device("cuda")

model_id = "LLM-LAT/zephyr7b-beta-rmu-lat-unlearn-wmdp-bio-cyber"
model_id = "meta-llama/Meta-Llama-3-8B"
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "google/gemma-2b"
model_id = "HuggingFaceH4/zephyr-7b-beta"
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_id, torch_dtype=torch.bfloat16, trust_remote_code=True, token=hf_access_token
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/mnt/align4_drive/data/huggingface/hub/models--HuggingFaceH4--zephyr-7b-beta/.no_exist/892b3d7a7b1cf10c7a701c60881cd93df615734c/adapter_config.json'
Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/mnt/align4_drive/data/huggingface/hub/models--HuggingFaceH4--zephyr-7b-beta/.no_exist/892b3d7a7b1cf10c7a701c60881cd93df615734c/adapter_config.json'


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [153]:
from unlearn_order.datasets.utils import DATASETS_DICT, Datasets


dataset_config = DATASETS_DICT[Datasets.WMDP_MCQ_CORPUS]

In [154]:
# ascent on corpus split, descent on wikitext
# eval loss on regular split mcq val split, wikitext val split
# also should eval on train split mcq val split, wikitext, mmlu cats
# all val files should be mcq
# all train should be corpus
# unless u do rtt

from datasets import load_dataset
from unlearn_order.dataset import load_dataset as load_dataset_unlearn

retain_dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

data_dir = Path("../data")

forget_train = load_dataset_unlearn(data_dir, dataset_config["unlearn_files"])
forget_val = load_dataset_unlearn(data_dir, dataset_config["val_files"])

retain_train = load_dataset("wikitext", "wikitext-2-raw-v1")["validation"]
retain_val = load_dataset_unlearn(data_dir, dataset_config["val_retain_files"])

In [155]:
retain_train

Dataset({
    features: ['text'],
    num_rows: 3760
})

In [156]:
from datasets import Dataset
from transformers import AutoTokenizer
from functools import partial


def map_corpus(data, tokenizer, max_length):
    text = data["text"]
    output = tokenizer(
        text,
        padding="max_length",
        max_length=max_length,
        truncation=True,
        return_tensors="pt",
    )
    return {
        "input_ids": output["input_ids"].squeeze(),
        "attention_mask": output["attention_mask"].squeeze(),
        "labels": output["input_ids"].clone().squeeze(),
    }


def process_train_dataset(
    dataset: Dataset,
    tokenizer: AutoTokenizer,
    max_length: int = 512,
    min_length: int = 0,
):
    dataset = dataset.filter(lambda x: len(x["text"]) > min_length)
    dataset = dataset.map(
        partial(map_corpus, tokenizer=tokenizer, max_length=max_length)
    )

    keep_cols = ["input_ids", "attention_mask", "labels"]
    dataset = dataset.remove_columns(
        [col for col in dataset.column_names if col not in keep_cols]
    )
    dataset.set_format("torch")
    return dataset


max_length = 64
forget_train = process_train_dataset(forget_train, tokenizer, max_length=max_length)
retain_train = process_train_dataset(retain_train, tokenizer, max_length=max_length)

Filter:   0%|          | 0/2355 [00:00<?, ? examples/s]

Map:   0%|          | 0/2355 [00:00<?, ? examples/s]

In [157]:
retain_val[1]
from unlearn_order.utils import (
    create_prompt_letter_answer,
    create_prompt,
    create_prompt_question_answer,
)

prompt = create_prompt_question_answer(retain_val[1])
# prompt = create_prompt(retain_val[1])
print(prompt)
print(retain_val[1])

Severe anaemia at birth is a feature of what? Alpha-thalassaemia
{'question': 'Severe anaemia at birth is a feature of what?', 'choices': ['Alpha-thalassaemia', 'Beta-thalassaemia', 'Hereditary persistence of fetal haemoglobin', 'Sickle cell disease'], 'answer': 0}


In [218]:
from copy import deepcopy
import torch.nn.functional as F

def create_mcq(record, tokenizer, max_length):
    text = create_prompt_letter_answer(record)
    prompt = create_prompt(record)
    # prompt += " "

    completion = text[len(prompt) :]

    record["prompt"] = prompt
    record["text"] = text
    record["completion"] = completion

    prompt_ids = tokenizer(prompt, return_tensors="pt").input_ids[0]
    completion_ids = tokenizer(completion, return_tensors="pt").input_ids[0]

    input_ids = torch.cat([prompt_ids, completion_ids])
    labels = input_ids.clone()

    attention_mask = torch.ones_like(input_ids)
    prompt_mask = torch.zeros_like(input_ids)
    prompt_mask[: len(prompt_ids)] = 1
    completion_mask = torch.zeros_like(input_ids)
    completion_mask[len(prompt_ids) :] = 1

    # pad to max length on left
    seq_len = input_ids.size(0)
    pad_len = max_length - seq_len
    padding = (0, pad_len)
    input_ids = F.pad(input_ids, padding, value=0)
    labels = F.pad(labels, padding, value=-100)
    attention_mask = F.pad(attention_mask, padding, value=0)
    prompt_mask = F.pad(prompt_mask, padding, value=0)
    completion_mask = F.pad(completion_mask, padding, value=0)

    record["input_ids"] = input_ids
    record["labels"] = labels
    record["attention_mask"] = attention_mask
    record["completion_byte_len"] = len(completion.encode("utf-8"))
    record["prompt_mask"] = prompt_mask
    record["completion_mask"] = completion_mask

    return record


def expand_mcq_records(records, **kwargs):
    new_records = []
    for rec in records:
        n_choices = len(rec["choices"])
        for i in range(n_choices):
            new_rec = deepcopy(rec)
            actual_answer = rec["answer"]
            new_rec["answer"] = i
            new_rec = create_mcq(new_rec, **kwargs)
            new_rec["answer"] = actual_answer
            new_rec["selected_answer"] = i
            new_records.append(new_rec)

    return new_records



In [219]:
max_length = 1024

forget_train_records = forget_train.to_list()
retain_train_records = retain_train.to_list()
forget_val_records = forget_val.to_list()
retain_val_records = retain_val.to_list()
retain_val_records = expand_mcq_records(retain_val_records, tokenizer=tokenizer, max_length=max_length)
forget_val_records = expand_mcq_records(forget_val_records, tokenizer=tokenizer, max_length=max_length)

In [226]:
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

def evaluate(model, records, batch_size=8, n_choices=4):
    # round up to nearest multiple of n_choices
    batch_size = (batch_size + n_choices - 1) // n_choices * n_choices 
    fields = ["input_ids", "attention_mask", "labels"]
    aux_fields = ["answer", "completion_byte_len", "completion_mask"]
    fields.extend(aux_fields)
    dataset = [ {k: v for k, v in rec.items() if k in fields} for rec in records]

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in tqdm(dataloader):
            # print(batch)
            for k, v in batch.items():
                if type(v) == torch.Tensor and k not in ["answer", "completion_byte_len"]:
                    assert torch.all(v[:, 111:113] == 0) or torch.all(v[:, 111:113] == -100)
                    batch[k] = v[:, :113]
                
            inputs = {k: v.to(device) for k, v in batch.items() if k not in aux_fields}
            outputs = model(**inputs)
            logits = outputs.logits

            labels = batch["labels"].to(device)
            completion_mask = batch["completion_mask"]
            attention_mask = inputs["attention_mask"]
            completion_byte_len = batch["completion_byte_len"].to(device)
            answers = batch["answer"].to(device)

            # compute completions loss
            labels[~completion_mask.bool()] = -100
            labels[~attention_mask.bool()] = -100

            logits = logits[:, :111]
            labels = labels[:, :111]

            shifted_logits = logits[:, :-1, :].contiguous()
            shifted_labels = labels[:, 1:].contiguous()

            loss = F.cross_entropy(shifted_logits.view(-1, shifted_logits.size(-1)), shifted_labels.view(-1), reduction="none")
            loss = loss.view(shifted_labels.size(0), shifted_labels.size(1))
            print(loss)
            loss_by_sample = loss[:, :111].sum(dim=1)


            print(loss_by_sample)
            loss_by_sample = loss_by_sample / completion_byte_len   
            loss_by_sample = loss_by_sample.view(-1, n_choices)
            
            answers = answers[::n_choices]
            print(answers)

            pred = loss_by_sample.argmin(dim=1)

            correct += (pred == answers).sum().item()
            total += len(answers)

    accuracy = correct / total
    return accuracy

start = 5
acc = evaluate(model, retain_val_records[start*4:start*4+4], batch_size=4)
print(acc)


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 15.32it/s]

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e

In [170]:
# retain_val_records[start*4:start*4+4]

In [175]:
from unlearn_order.eval import eval_dataset as my_eval_dataset

use_val = retain_val.select([start])
# use_val = retain_val.select(range(100))
effective_batch_size = 1
batch_size = effective_batch_size * 4

acc = my_eval_dataset(model, tokenizer, use_val, batch_size=batch_size)
print(acc)

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]/mnt/align1_drive/tcqian/unlearning_order/src/unlearn_order/eval.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  byte_lengths = torch.tensor(batch["byte_length"], device=model.device)
Evaluating: 100%|██████████| 1/1 [00:00<00:00, 15.32it/s]

torch.Size([4, 111, 32000])
tensor([47.8315, 38.9943, 46.9212, 44.8689], device='cuda:0')
1.0


In [ ]:
print(acc)

In [ ]:
from torch.utils.data import Dataset
from typing import Dict, Union, List, Optional


class ForgetRetainDataset(Dataset):
    def __init__(self, forget_dataset, retain_dataset):
        self.forget_dataset = forget_dataset
        self.retain_dataset = retain_dataset

    def __len__(self):
        min_len = min(len(self.forget_dataset), len(self.retain_dataset))
        return min_len

    def __getitem__(self, idx):
        return {
            "forget": self.forget_dataset[idx % len(self.forget_dataset)],
            "retain": self.retain_dataset[idx % len(self.retain_dataset)],
        }

In [8]:
from torch.utils.data import DataLoader
from transformers import Trainer, TrainingArguments
from unlearn_order.utils import log_1_minus_p_loss, my_loss
from unlearn_order.eval import eval_dataset as my_eval_dataset

from transformers import TrainerCallback, Trainer, TrainingArguments


class EvalCallback(TrainerCallback):
    def __init__(self, eval_steps=None, eval_epochs=False, eval_on_start=False):
        super().__init__()
        self.eval_steps = eval_steps
        self.eval_epochs = eval_epochs
        self.eval_on_start = eval_on_start

    def on_step_end(self, args, state, control, **kwargs):
        # Trigger evaluation every N steps
        if self.eval_steps and state.global_step % self.eval_steps == 0:
            control.should_evaluate = True

    def on_epoch_end(self, args, state, control, **kwargs):
        # Trigger evaluation at the end of every epoch
        if self.eval_epochs:
            control.should_evaluate = True


class GradientAscentTrainer(Trainer):
    def __init__(
        self,
        *args,
        forget_alpha: float = 1,
        forget_val: Dataset,
        retain_val: Dataset,
        **kwargs,
    ):
        super().__init__(*args, **kwargs)
        self.forget_alpha = forget_alpha
        self.forget_val = forget_val
        self.retain_val = retain_val

    def get_train_dataloader(self):
        return DataLoader(
            self.train_dataset, batch_size=self.args.per_device_train_batch_size
        )

    def compute_loss(self, model, inputs):
        """
        Compute the loss for a single batch of training data.
        """
        torch.cuda.empty_cache()
        # print(f"1, Cuda memory allocated: {torch.cuda.memory_allocated() / 1e9} GB")
        forget_inputs = {k: v.to(self.args.device) for k, v in inputs["forget"].items()}
        retain_inputs = {k: v.to(self.args.device) for k, v in inputs["retain"].items()}

        # forget_loss = my_loss(model, is_away=True, **forget_inputs)
        # retain_loss = my_loss(model, is_away=False, **retain_inputs)
        retain_loss = model(**forget_inputs).loss
        loss = retain_loss
        # print(f"2, Cuda memory allocated: {torch.cuda.memory_allocated() / 1e9} GB")

        # loss = self.forget_alpha * forget_loss + retain_loss

        return loss

    def custom_evaluate(self):
        forget_val_acc = my_eval_dataset(
            self.model,
            self.tokenizer,
            self.forget_val,
            batch_size=self.args.per_device_eval_batch_size,
        )
        torch.cuda.empty_cache()

        retain_val_acc = my_eval_dataset(
            self.model,
            self.tokenizer,
            self.retain_val,
            batch_size=self.args.per_device_eval_batch_size,
        )
        return forget_val_acc, retain_val_acc

    def evaluate(
        self,
        eval_dataset: Optional[Union[Dataset, Dict[str, Dataset]]] = None,
        ignore_keys: Optional[List[str]] = None,
        metric_key_prefix: str = "eval",
    ):
        # outs = super().evaluate(eval_dataset, ignore_keys, metric_key_prefix)
        outs = {}
        outs["forget_val_acc"], outs["retain_val_acc"] = self.custom_evaluate()
        outs = {f"{metric_key_prefix}_{key}": value for key, value in outs.items()}
        print(outs)
        return outs


def data_collator(batch):
    batch = {
        batch_id: {key: torch.stack(value) for key, value in batch_items.items()}
        for batch_id, batch_items in batch.items()
    }
    return batch


batch_size = 4

combined_dataset = ForgetRetainDataset(forget_train, retain_train)


# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",  # output directory
    num_train_epochs=3,  # total number of training epochs
    per_device_train_batch_size=batch_size
    // 4,  # batch size per device during training
    per_device_eval_batch_size=16,
    logging_dir="./logs",  # directory for storing logs
    logging_steps=100,
    save_strategy="no",
    evaluation_strategy="steps",
    eval_steps=100,
    eval_on_start=True,
    gradient_accumulation_steps=4,
    bf16=True,
    torch_empty_cache_steps=4,  # TODO(tcqian): why do i need this?
)

# Initialize Trainer
trainer = GradientAscentTrainer(
    model=model,
    args=training_args,  # Training arguments (e.g., learning rate, epochs)
    train_dataset=combined_dataset,
    eval_dataset=combined_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    forget_alpha=0.0,
    forget_val=forget_val,
    retain_val=retain_val,
    callbacks=[EvalCallback(eval_epochs=True, eval_on_start=True)],
)

# trainer.custom_evaluate()

# Train
trainer.train()

/mnt/align1_drive/tcqian/unlearning_order/venv/lib/python3.12/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


[2024-11-17 18:56:07,393] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/mnt/align1_drive/tcqian/miniconda/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/mnt/align1_drive/tcqian/miniconda/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/mnt/align1_drive/tcqian/miniconda/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/mnt/align1_drive/tcqian/miniconda/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/mnt/align1_drive/tcqian/miniconda/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/mnt/align1_drive/tcqian/miniconda/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/mnt/align1_dri

Evaluating:  24%|██▍       | 47/197 [00:08<00:58,  2.58it/s]

: 

: 